# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by Joseph Bekhit

## Table of contents
* [Introduction: Business Problem](#introduction1)
* [Data](#data1)
* [Methodology](#methodology1)
* [Results and Discussion](#results1)
* [Conclusion](#conclusion1)



## Introduction: Business Problem <a name="introduction1"></a>

New York,the financial capital of USA and so it is attractive to alot of investors who wants to open new business.In this project I will try to find an optimal location for a new business. Specifically, this report will be targeted to stakeholders interested in opening a new business in New York.I will list the neighborhoods in New York , the category of each business that exist in New York and the number of existing businesses of each category grouped by  New York neighborhoods.I will create a list of the all New York neighborhoods and the recommended business to open in that neighborhood.I will also create a list  of all the business categories and the recommended neighborhood to open that business in.Of course the second list make more sense than the first one because it is more realistic   in the real life that the stakeholder has  a particular business and he is looking for the best place where to open it.


## Data <a name="data1"></a>

I used (https://cocl.us/new_york_dataset) to get a json file of the neighborhoods of New York with the longitude and latitude of each neighborhood.Then I used foursquare API to get the venues that exist in each neighborhood.The data returned from foursquare include the venue name of each venue that exist  and it's category.


In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim #convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    nigh = json.load(json_data)
neigh = nigh['features']
# define the dataframe columns
nighcolumn = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=nighcolumn)
for data in neigh:
    borough =data['properties']['borough'] 
    neighborhood_name = data['properties']['name']        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
print(neighborhoods.head())
print(neighborhoods.shape)

  Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585
(306, 4)


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


In [14]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [15]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)    
map_newyork

In [16]:
CLIENT_ID = 'ZLU2POEDLOQSMY2KKNAWU2OXAY5BASJDS0F5FYIGRHGR1KPT' # my Foursquare ID
CLIENT_SECRET = 'KTVCBEEFUWQWSFQPAW5TECFO3K4JAMHVRRHP3B3XEMKS25OK' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('my credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

my credentails:
CLIENT_ID: ZLU2POEDLOQSMY2KKNAWU2OXAY5BASJDS0F5FYIGRHGR1KPT
CLIENT_SECRET:KTVCBEEFUWQWSFQPAW5TECFO3K4JAMHVRRHP3B3XEMKS25OK


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

## Methodology <a name="methodology1"></a>

In this project we will direct our efforts on detecting the areas of New York with the business that exist in each area grouped by the business category. 
In first step we have collected the required data: location and type (category) of every business in every neighborhood in New York.We did that according to Foursquare categorization.
Second step in our analysis will be calculation and exploration of the count of each business across the different neighborhoods in New York.The stakeholder can then investigate the result and choose the business category with the count equal zero or as minimum as possible,or choose the neighborhood with the minimum count of the business category that he is interested in.

In [18]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Wakefield


KeyError: 'groups'

In [12]:
print(newyork_venues.shape)
newyork_venues.head()

NameError: name 'newyork_venues' is not defined

In [ ]:
newyork_venues.groupby('Neighborhood').count()

In [13]:
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 431 uniques categories.


In [14]:
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]
newyork_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaur

In [ ]:
newyork_onehot.shape

In [ ]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').sum().reset_index()
newyork_grouped.head()

In [17]:
newyork_grouped.shape

(301, 431)

In [ ]:
num_top_venues =10 #len(newyork_venues['Venue Category'].unique())
for hood in newyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =newyork_grouped[newyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
newyork_onehot2 = pd.get_dummies(newyork_venues[['Neighborhood']], prefix="", prefix_sep="")
newyork_onehot2['Venue Category'] = newyork_venues['Venue Category'] 
fixed_columns = [newyork_onehot2.columns[-1]] + list(newyork_onehot2.columns[:-1])
newyork_onehot2 = newyork_onehot2[fixed_columns]
newyork_onehot2.head()

In [ ]:
newyork_onehot2.shape

In [ ]:
newyork_grouped2 = newyork_onehot2.groupby('Venue Category').sum().reset_index()
newyork_grouped2.head()

In [29]:
newyork_grouped2.shape

(431, 302)

In [ ]:
num_bottom_venues =10 #len(newyork_venues['Venue Category'].unique())
for category in newyork_grouped2['Venue Category']:
    print("----"+category+"----")
    temp =newyork_grouped2[newyork_grouped2['Venue Category'] == category].T.reset_index()
    temp.columns = ['Neighborhood','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    print(temp.sort_values('freq', ascending=True).reset_index(drop=True).head(num_bottom_venues))
    print('\n')

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
def return_least_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']
for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Spa,Chinese Restaurant,Donut Shop,Playground,Cosmetics Shop,Electronics Store,Pharmacy
1,Annadale,Pizza Place,Diner,Train Station,Sports Bar,American Restaurant,Pharmacy,Restaurant,Food,Bakery,Sushi Restaurant
2,Arden Heights,Deli / Bodega,Pizza Place,Bus Stop,Coffee Shop,Pharmacy,Dim Sum Restaurant,Eye Doctor,Falafel Restaurant,Farm,Farmers Market
3,Arlington,Boat or Ferry,Grocery Store,Bus Stop,Intersection,American Restaurant,Coffee Shop,Farm,Farmers Market,Fast Food Restaurant,Field
4,Arrochar,Bus Stop,Italian Restaurant,Deli / Bodega,Pizza Place,Polish Restaurant,Bagel Shop,Liquor Store,Athletics & Sports,Supermarket,Hotel


In [36]:
num_bottom_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']
for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_least_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Yoga Studio,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant,Paella Restaurant,Outlet Store
1,Annadale,Yoga Studio,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant,Paella Restaurant,Outlet Store,Outdoors & Recreation
2,Arden Heights,Yoga Studio,Pet Café,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant
3,Arlington,Yoga Studio,Pet Service,Pet Café,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store
4,Arrochar,Yoga Studio,Pet Café,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant


In [37]:
kclusters = 5
newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 3, 1, 1, 0], dtype=int32)

In [38]:
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)
newyork_merged = neighborhoods
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [39]:
newyork_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1.0,Yoga Studio,Pet Café,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,1.0,Yoga Studio,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Paper / Office Supplies Store,Pakistani Restaurant,Paella Restaurant,Outlet Store
2,Bronx,Eastchester,40.887556,-73.827806,1.0,Yoga Studio,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant,Paella Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1.0,Yoga Studio,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant,Paella Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,1.0,Yoga Studio,Pet Service,Pet Café,Peruvian Restaurant,Persian Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Paper / Office Supplies Store,Pakistani Restaurant


In [40]:
i=0
while i<newyork_merged.shape[0]:
    x=newyork_merged["Cluster"][i]
    if  x!=0 and x!=1 and x!=2 and x!=3 and x!=4:
        newyork_merged["Cluster"][i]=0     
        print(newyork_merged["Cluster"][i])
    i=i+1    

0.0


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [ ]:
print(newyork_merged.loc[newyork_merged['Cluster'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]])
print("#######################END OF CLUSTER 1##################################")
print(newyork_merged.loc[newyork_merged['Cluster'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]])
print("#######################END OF CLUSTER 2##################################")
print(newyork_merged.loc[newyork_merged['Cluster'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]])
print("#######################END CLUSTER 3##################################")
print(newyork_merged.loc[newyork_merged['Cluster'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]])
print("#######################END OF CLUSTER 4##################################")
print(newyork_merged.loc[newyork_merged['Cluster'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]])
print("#######################END OF CLUSTER 5##################################")

## Results and Discussion <a name="results1"></a>

we found that there are many business categories that does not exist in many neighborhoods,where these business categories  exist strongly in other neighborhoods, and these business categories are recommended to be opened in these neighborhoods.If you are a new investor and do not have a specific business category you can choose the business category the does not exist in the favorite neighborhood and if you have a specific business category to open,you can choose the neighborhood which you business category does not exist.


## Conclusion <a name="conclusion1"></a>

Although New York is  old  and crowded state,it is still attractive  for investors because it has many business categories that exist in some neightborhoods,while does not exist at all at some other neighborhoods.
